# Create OMMX Artifact

In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
import uuid
from ommx.artifact import Artifact, ArtifactBuilder
from ommx.testing import SingleFeasibleLPGenerator, DataType

In [3]:
generator = SingleFeasibleLPGenerator(3, DataType.INT)
instance = generator.get_v1_instance()
state = generator.get_v1_state()

In [4]:
builder = ArtifactBuilder.for_github(
    "Jij-Inc",             # GitHub organization
    "ommx",                # Repository name
    "single_feasible_lp",  # Name of artifact
    str(uuid.uuid4())      # Tag of artifact
)
builder.add_instance(instance)
artifact = builder.build()

In [5]:
artifact.push()

INFO:ommx.artifact:Pushing: ghcr.io/jij-inc/ommx/single_feasible_lp:df5f1788-9ae9-4cd6-8ee0-0bfdfed56fc0
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/7121c776-0087-4e77-8c8f-010f9e6f838f
INFO:ocipkg.distribution.client:POST https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/uploads/
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/blobs/upload/a1026342-6eb8-4ad2-bc69-3a6d57b2a1c1
INFO:ocipkg.distribution.client:PUT https://ghcr.io/v2/jij-inc/ommx/single_feasible_lp/manifests/df5f1788-9ae9-4cd6-8ee0-0bfdfed56fc0
